In [1]:
from operator import itemgetter
from src.utils import DataParser, utils
import pandas as pd
from src.detections import Detector, Tracker
from ultralytics import YOLO

In [6]:
directory = '../data/SoccerNetGS/gamestate-2024/test'
data_parser = DataParser(directory)
df = data_parser.parse_data_to_df()

Wczytywanie danych...


100%|██████████| 50/50 [00:09<00:00,  5.37it/s]


Tworzenie DataFrame...


100%|██████████| 49/49 [00:03<00:00, 15.55it/s]


In [125]:
frames_numes = df.image_id.unique()

In [7]:
df_one_match = df[df['image_id'].str.startswith('3118')]
df_one_match = df_one_match[df_one_match['attributes_role'] == 'player']

In [10]:
df_few_frames = df_one_match[df_one_match['image_id'].isin(frames[:10])]
df_few_frames

,image_id,track_id,supercategory,category_id,attributes_role,attributes_jersey,attributes_team,bbox_image_x,bbox_image_y,bbox_image_x_center,...,bbox_pitch_x_bottom_right,bbox_pitch_y_bottom_right,bbox_pitch_x_bottom_middle,bbox_pitch_y_bottom_middle,bbox_pitch_raw_x_bottom_left,bbox_pitch_raw_y_bottom_left,bbox_pitch_raw_x_bottom_right,bbox_pitch_raw_y_bottom_right,bbox_pitch_raw_x_bottom_middle,bbox_pitch_raw_y_bottom_middle
3116000001,3116000001,1,object,1,player,4,left,136,520,161.5,...,29.723592,-4.962108,29.486149,-5.143334,29.080245,-5.217584,29.589248,-4.830455,29.334593,-5.024136
3116000002,3116000001,2,object,1,player,93,left,478,565,503.5,...,30.964130,0.980532,30.722999,0.793530,30.467952,0.617342,30.946138,0.986441,30.706910,0.801787
3116000003,3116000001,3,object,1,player,25,right,565,518,589.0,...,34.566522,-2.456439,34.312718,-2.649419,34.005048,-2.839869,34.504631,-2.461507,34.254693,-2.650799
3116000004,3116000001,4,object,3,referee,None,None,468,433,491.0,...,38.742209,-11.349614,38.466332,-11.551332,38.167492,-11.767518,38.734982,-11.353659,38.451050,-11.560725
3116000005,3116000001,5,object,1,player,None,left,557,302,574.0,...,54.752061,-33.012322,54.453909,-33.207236,54.134561,-33.394207,54.753082,-32.989247,54.443607,-33.191867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116000214,3116000010,16,object,1,player,36,right,1480,448,1494.5,...,49.570256,-0.363327,49.386397,-0.499578,49.223976,-0.620855,49.588175,-0.351031,49.405995,-0.486003
3116000215,3116000010,17,object,1,player,36,left,1493,446,1504.0,...,49.722132,-0.321507,49.574669,-0.430787,49.387158,-0.499959,49.663608,-0.295145,49.525336,-0.397587
3116000216,3116000010,18,object,1,player,8,right,1505,445,1526.5,...,50.175002,-0.129646,49.919168,-0.319052,49.621741,-0.466771,50.163933,-0.065263,49.892657,-0.266150
3116000217,3116000010,19,object,2,goalkeeper,None,right,1532,445,1552.0,...,51.092456,-0.396767,50.830936,-0.589779,50.562586,-0.769619,51.075101,-0.391339,50.818683,-0.580597


In [8]:
with open('gt.txt', 'w') as f:
    for row in df_one_match.itertuples():
        f.write(f"{int(row.image_id[-3:])},{row.track_id},{row.bbox_image_x:.2f},{row.bbox_image_y:.2f},{row.bbox_image_w:.2f},{row.bbox_image_h:.2f},1,-1,-1,-1\n")

In [10]:
gts = []
for row in df_one_match.itertuples():
    gts.append([int(row.image_id[-3:]), row.bbox_image_x, f"{int(row.image_id[-3:])},{row.track_id},{row.bbox_image_x:.2f},{row.bbox_image_y:.2f},{row.bbox_image_w:.2f},{row.bbox_image_h:.2f},1,-1,-1,-1\n"])

In [11]:
with open('gt.txt', 'w') as f:
    for gt in sorted(gts, key=itemgetter(0,1)):
        f.write(gt[2])

In [142]:
from operator import itemgetter
sorted(gts, key=itemgetter(0,1))

[[1, 46, '1,6,46,542,97,174,1,-1,-1,-1\n'],
 [1, 165, '1,3,165,403,69,122,1,-1,-1,-1\n'],
 [1, 208, '1,12,208,256,39,95,1,-1,-1,-1\n'],
 [1, 359, '1,4,359,222,45,86,1,-1,-1,-1\n'],
 [1, 638, '1,7,638,600,89,182,1,-1,-1,-1\n'],
 [1, 840, '1,5,840,251,44,100,1,-1,-1,-1\n'],
 [1, 968, '1,8,968,370,62,104,1,-1,-1,-1\n'],
 [1, 1000, '1,1,1000,791,73,202,1,-1,-1,-1\n'],
 [1, 1639, '1,2,1639,426,47,140,1,-1,-1,-1\n'],
 [2, 47, '2,6,47,540,97,174,1,-1,-1,-1\n'],
 [2, 167, '2,3,167,401,70,123,1,-1,-1,-1\n'],
 [2, 212, '2,12,212,255,39,95,1,-1,-1,-1\n'],
 [2, 359, '2,4,359,220,45,87,1,-1,-1,-1\n'],
 [2, 639, '2,7,639,597,89,182,1,-1,-1,-1\n'],
 [2, 841, '2,5,841,250,44,100,1,-1,-1,-1\n'],
 [2, 974, '2,8,974,368,64,104,1,-1,-1,-1\n'],
 [2, 996, '2,1,996,792,78,202,1,-1,-1,-1\n'],
 [2, 1638, '2,2,1638,424,47,140,1,-1,-1,-1\n'],
 [3, 49, '3,6,49,538,97,174,1,-1,-1,-1\n'],
 [3, 169, '3,3,169,399,70,124,1,-1,-1,-1\n'],
 [3, 216, '3,12,216,253,39,95,1,-1,-1,-1\n'],
 [3, 359, '3,4,359,217,45,88,1,-1,-1

In [134]:
frames = utils.load_frames_from_images('../data/SoccerNetGS/gamestate-2024/test/SNGS-118/img1')

In [147]:
model = YOLO("../training/runs/detect/train4/weights/best.pt")
detector = Detector(model, 0.5)
tracker = Tracker(detector)

In [148]:
tracks = tracker.track(frames)


0: 384x640 10 players, 1 ball, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 11.5ms
Speed: 2.0ms preprocess, 11.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 1 ball, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 1 ball, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 1 ball, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 1 ball, 10.1ms
Speed: 2.0ms preprocess, 10.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 players, 11.0ms
Speed: 1.0ms preprocess, 11.0ms i

In [149]:
tracks[0]

Detections(xyxy=array([[     166.14,       404.4,      230.48,      523.41],
       [     643.25,      605.19,      729.06,      773.58],
       [     1001.8,      791.81,      1074.7,      995.21],
       [     212.41,      255.53,      248.58,      349.42],
       [     968.76,      370.25,      1027.4,      468.86],
       [     49.656,      545.92,       137.1,      711.37],
       [     845.81,      251.03,      882.92,      349.02],
       [       1639,      425.39,      1685.5,      564.24],
       [     364.79,       223.5,      401.84,         307],
       [     1517.6,      514.59,      1535.4,       532.1],
       [     1625.3,      186.06,      1674.7,      236.27]], dtype=float32), mask=None, confidence=array([    0.81455,     0.76636,     0.76083,     0.75708,      0.7555,       0.753,     0.75115,     0.73421,     0.70463,     0.68529,     0.55973], dtype=float32), class_id=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]), tracker_id=array([652, 653, 654, 655, 656, 657, 658, 659

In [164]:
with open('pred.txt', 'w') as f:
    for i, track in enumerate(tracks):
        try:
            classes = track.data['class_name']
        except KeyError:
            continue
        preds = []
        for xyxy, confidence, tracker_id, class_name in zip(track.xyxy, track.confidence, track.tracker_id, classes):
            w = round(xyxy[2] - xyxy[0], 2)
            h = round(xyxy[3] - xyxy[1], 2)
            if class_name == 'player':
                preds.append([xyxy[0], f"{i+1},{tracker_id},{round(float(xyxy[0]),2)},{round(float(xyxy[1]),2)},{round(float(w),2)},{round(float(h),2)},{round(float(confidence),2)},-1,-1,-1\n"])
        for pred in sorted(preds, key=itemgetter(0)):
            f.write(pred[1])

In [59]:
import motmetrics as mm
import numpy as np

gt = mm.io.loadtxt('gt.txt')
pred = mm.io.loadtxt('pred.txt')

acc = mm.MOTAccumulator(auto_id=True)
for frame in range(750):
    frame += 1 # detection and frame numbers begin at 1

    # select id, x, y, width, height for current frame
    # required format for distance calculation is X, Y, Width, Height \
    # We already have this format
    try:
        gt_dets = gt.loc[frame].to_numpy() # select all detections in gt 
    except KeyError:
        continue
    t_dets = pred.loc[frame].to_numpy() # select all detections in t
    C = mm.distances.iou_matrix(gt_dets[:,:4], t_dets[:,:4],  max_iou=0.5) # format: gt, t

    # Call update once for per frame.
    # format: gt object ids, t object ids, distance
    acc.update(gt.loc[frame].index.astype('int').to_list(), pred.loc[frame].index.astype('int').to_list(), C)

mh = mm.metrics.create()

summary = mh.compute(acc, metrics=['num_frames', 'idf1', 'idp', 'idr', \
                                     'recall', 'precision', 'num_objects', \
                                     'mostly_tracked', 'partially_tracked', \
                                     'mostly_lost', 'num_false_positives', \
                                     'num_misses', 'num_switches', \
                                     'num_fragmentations', 'mota', 'motp' \
                                    ], \
                      name='acc')

strsummary = mm.io.render_summary(
  summary,
  #formatters={'mota' : '{:.2%}'.format},
  namemap={'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', \
           'precision': 'Prcn', 'num_objects': 'GT', \
           'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
           'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
           'num_misses': 'FN', 'num_switches' : 'IDsw', \
           'num_fragmentations' : 'FM', 'mota': 'MOTA', 'motp' : 'MOTP',  \
          }
)
print(strsummary)


     num_frames      IDF1       IDP       IDR      Rcll      Prcn    GT  MT  PT  ML   FP   FN  IDsw   FM      MOTA      MOTP
acc         725  0.835097  0.867886  0.804696  0.910827  0.982351  8433  13   4   3  138  752    28  104  0.891142  0.180765


In [42]:
gt.loc[1].to_numpy()[:,0]

array([         45,         164,         207,         358,         637,         839,         967,         999,        1638])

In [56]:
gt

X      Y  Width  Height  Confidence  ClassId  Visibility
FrameId Id                                                               
1       6     45.0  541.0   97.0   174.0           1       -1          -1
        3    164.0  402.0   69.0   122.0           1       -1          -1
        12   207.0  255.0   39.0    95.0           1       -1          -1
        4    358.0  221.0   45.0    86.0           1       -1          -1
        7    637.0  599.0   89.0   182.0           1       -1          -1
...            ...    ...    ...     ...         ...      ...         ...
750     7    445.0  728.0  104.0   221.0           1       -1          -1
        5    785.0  214.0   51.0   106.0           1       -1          -1
        8    920.0  448.0   84.0   154.0           1       -1          -1
        2   1474.0  483.0   91.0   168.0           1       -1          -1
        1   1501.0  729.0  103.0   212.0           1       -1          -1

[8433 rows x 7 columns]

In [ ]:
gt[:,0]

In [14]:
acc.events['Type'].unique()

['RAW', 'MISS', 'FP']
Categories (8, object): ['RAW', 'FP', 'MISS', 'SWITCH', 'MATCH', 'TRANSFER', 'ASCEND', 'MIGRATE']